In [62]:
import numpy as np

import onnx
from onnx import numpy_helper, TensorProto, checker
from onnx import helper as onnx_helper


In [63]:
model = onnx.load('without_nms.onnx')
graph = model.graph

In [64]:
names = [output.name for output in graph.output]
names

['output', '298', '316']

In [65]:
value = np.array([1, -1, 10], dtype=np.int64)
reshape_size_1 = numpy_helper.from_array(value, 'reshape_size_1')
output_reshaped = onnx_helper.make_tensor_value_info(
    'output_reshaped',
    TensorProto.FLOAT, [1, None, -4]
)

node = onnx_helper.make_node(
    'Reshape',
    ['output', 'reshape_size_1'],
    ['output_reshaped']
)

graph.initializer.append(reshape_size_1)
graph.node.append(node)


In [66]:
while graph.output:
    graph.output.pop()
graph.output.append(output_reshaped)


In [67]:
checker.check_model(model)

In [68]:
onnx.save(model, 'test.onnx')

## Test that model make inference

In [69]:
import onnxruntime as onnx_run
import cv2


In [70]:
session = onnx_run.InferenceSession('test.onnx')
session

In [77]:
image = cv2.imread('DJI_0418-00000017.JPG')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# crop image
height, width = image.shape[:2]
startX = width // 2 - 320
startY = height // 2 - 320
endX = startX + 640
endY = startY + 640
image = image[startY:endY, startX:endX]

batch = np.moveaxis(image, -1, 0)[None, ...] / np.float32(255)

result = session.run(None, {'images': batch})
result = result[0]

In [81]:
result.shape

(1, 19200, 10)

In [83]:
result[0, 0, :]

array([ 0.19433665,  0.0771229 , -0.78236604,  0.60730326, -7.3969955 ,
       -3.5028577 , -3.6214676 , -0.46629   ,  0.74715555, -3.5403032 ],
      dtype=float32)